In [19]:
# Install dependencies
!pip install networkx matplotlib pygraphviz

In [20]:
import os

import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
from IPython.display import Image
from networkx.drawing.nx_agraph import to_agraph
from random import choice

# Ensure the assets directory exists
os.makedirs('assets', exist_ok=True)

prefix = "aufgabe_02"
asset_dir = os.path.join('assets', prefix)
os.makedirs(asset_dir, exist_ok=True)

# Aufgabe 02

In [26]:
transactions = [
    ('R', 1, 'A'),
    ('W', 1, 'A'),
    ('R', 7, 'B'),
    ('R', 2, 'A'),
    ('W', 2, 'A'),
    ('R', 1, 'B'),
    ('W', 1, 'B'),
    ('R', 2, 'B'),
    ('W', 2, 'B'),
    ('R', 3, 'A'),
    ('W', 4, 'A'),
    ('R', 5, 'B'),
    ('R', 1, 'C'),
    ('R', 6, 'C'),
    ('W', 8, 'A'),
    ('W', 8, 'B')
]

graph = nx.DiGraph()
graph.add_edges_from([(1, 2), (1, 3), (1, 4), (1, 5), (1, 8), (2, 3), (2, 4), (2, 5), (2, 8), (5, 8), (7, 1)])

# Visualize the graph using PyGraphViz
agraph = to_agraph(graph)
filename = os.path.join(asset_dir, f"dependency_graph.png")
agraph.draw(filename, format="png", prog="dot")
display(Image(filename=filename))

serial_schedule, cycle_exists